In [0]:
import logging
from pyspark.sql import functions as F

logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)

class SaveHelper:
    def __init__(self, df, layer, table_path, mode):
        """
        Classe utilitária para fisicalizar dados.

        Args:
            df (spark.DataFrame): DataFrame com dados
            layer (str): Camada para salvar
            table_path (str): Caminho da tabela
            mode (str): Modo de salvamento dos dados (append or overwrite)

        Raises:
            ValueError: Se os parâmetros forem inválidos
        """
        self._validate_inputs(df, layer, table_path, mode)

        self.df = df
        self.layer = layer.lower()
        self.table_path = table_path
        self.table_exists = spark.catalog.tableExists(self.table_path)
        self.mode = mode.lower()

        logger.info(f"SaveHelper inicializado - Tabela: {table_path} - Mode: {mode} - Layer: {layer}")

    def _validate_inputs(self, df, layer, table_path, mode):
        """Valida os parâmetros de entrada"""
        if df is None:
            raise ValueError("DataFrame não pode ser None")
        
        if not hasattr(df, 'write'):
            raise ValueError(f"Esperado DataFrame do Spark, recebido: {type(df)}")
        
        if df.count() == 0:
            logger.warning("DataFrame está vazio!")
        
        valid_modes = ["append", "overwrite"]
        if mode.lower() not in valid_modes:
            raise ValueError(f"Mode deve ser um entre as opções: {valid_modes}")

        valid_layers = ["bronze", "silver", "gold"]
        if layer.lower() not in valid_layers:
            raise ValueError(f"Layer deve ser uma entre as opções: {valid_layers}")

        if not table_path or "." not in table_path:
            raise ValueError("O table_path deve seguir formato 'catalog.schema.table'")

    def _create_table_if_not_exists(self):
        """Cria tabela se não existir"""
        if not self.table_exists:
            try:
                spark.catalog.createTable(self.table_path, source="delta", schema=self.df.schema)
                logger.info(f"✅ Tabela criada com sucesso: {self.table_path}")
            except Exception as e:
                logger.error(f"❌ Erro ao criar a tabela: {self.table_path}")

    def _add_metadata(self):
        """ Adiciona data_ingestao no DataFrame """
        if "data_ingestao" not in self.df.columns:
            self.df = self.df.withColumn("data_ingestao", F.current_date())

    def _saveTable(self):
        """ Salva a tabela de acordo com o mode informado """
        try:
            if self.mode == "append":
                self.df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(self.table_path)
            elif self.mode == "overwrite":
                self.df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(self.table_path)
            logger.info(f"✅ {self.df.count()} registros inseridos com sucesso!")
        except Exception as e:
            logger.error(f"❌ Erro ao salvar os dados na tabela {self.table_path}: {e}")

    def execute(self):
        self._create_table_if_not_exists()
        self._add_metadata()
        self._saveTable()